In [ ]:
from functools import cache

EXAMPLE = "../example.txt"
INPUT = "../input.txt"

In [ ]:
def parse_input(input_file_name):
    towels = set()
    designs = []
    reading_towels = True
    with open(input_file_name, 'r') as f:
        for line in f:
            if line == '\n':
                reading_towels = False
                continue
            if reading_towels:
                for towel in line.strip().replace('\n', '').split(','):
                    towels.add(towel.strip())
                continue
            else:
                designs.append(line.strip().replace('\n', ''))
    return towels, designs

In [ ]:
towels, designs = parse_input(EXAMPLE)
print(towels)
print(designs)

For any given design, we go through all the towels to find the ones that are a prefix to the design.

Then we remove the prefix and add the result to our list of next designs we'll need to check.

In [ ]:
def get_next_designs(towels, design):
    next_designs = []
    for towel in towels:
        if design.startswith(towel):
            next_designs.append(design[len(towel):])
    return next_designs

Let's create a Towels class to be able to cache the calls to get_next_designs, it should allow us to speed up the runs since the same designs can come up many different times.

In [ ]:
class Towels:
    towels: set

    def __init__(self, towels):
        self.towels = towels

    @cache
    def get_next_designs(self, design):
        next_designs = []
        for towel in self.towels:
            if design.startswith(towel):
                next_designs.append(design[len(towel):])
        return next_designs

In [ ]:
towels = Towels(towels)

In [ ]:
def design_is_possible(towels, design):
    designs = set([design])
    while designs:
        new_designs = set()
        for design in designs:
            # Find the designs that have a prefix compatible with the towels and get their tails
            next_designs = towels.get_next_designs(design)
            if '' in next_designs:
                # The tail is empty, we've successfully reached the end of the design
                return True
            for next_design in next_designs:
                # Update the designs we need to check in the next step
                new_designs.add(next_design)
        designs = new_designs
    return False

In [ ]:
for design in designs:
    print(design_is_possible(towels, design))

In [ ]:
def part_1(input_file_name):
    towels, designs = parse_input(input_file_name)
    towels = Towels(towels)
    result = 0
    for design in designs:
        if design_is_possible(towels, design):
            result += 1
    print(result)

In [ ]:
part_1(EXAMPLE)

In [ ]:
part_1(INPUT)

In [ ]:
def nb_of_arrangements(towels, design):
    # Keep track of the number of times we've been able to build any design
    designs = {design: 1}
    result = 0
    while designs:
        new_designs = {}
        for design, count in designs.items():
            # For each unique design, we call the function only once
            next_designs = towels.get_next_designs(design)
            # Next designs contains the tails reachable from the current design
            # Each one is reachable in "count" different ways
            for next_design in next_designs:
                if next_design == '':
                    # We've successfully reached the end of a design in "count" different ways
                    # Add it to the total
                    result += next_designs.count('')*count
                    continue
                # Update our design dict for the next step
                if next_design in new_designs:
                    new_designs[next_design] += count
                else:
                    new_designs[next_design] = count
        # Move on to the next step
        designs = new_designs
    return result

In [ ]:
def part_2(input_file_name):
    towels, designs = parse_input(input_file_name)
    towels = Towels(towels)
    result = 0
    for design in designs:
        nb = nb_of_arrangements(towels, design)
        result += nb
    print(result)

In [ ]:
part_2(EXAMPLE)

In [ ]:
part_2(INPUT)